In [1]:
import numpy as np
import math as math
import matplotlib.pyplot as plt

In [2]:
#original sequence
a = np.array([[0,1,"blue"],[0,2,"red"],[0,3,"blue"],[0,4,"red"],[0,5,"blue"]])
a

array([['0', '1', 'blue'],
       ['0', '2', 'red'],
       ['0', '3', 'blue'],
       ['0', '4', 'red'],
       ['0', '5', 'blue']], dtype='<U11')

i decided to work not with blue or red but with int so that the datatype will not be string

In [3]:
#converting sequence: blue to 1, red to 0
#arrangement of elements in each list: (x_coord, y_coord, label: H = 1, P = 0)
#starting sequence: HPPHPH
a = np.array([[0,1,1],[0,2,0],[0,3,0],[0,4,1],[0,5,0], [0,6,1]])

In [4]:
a

array([[0, 1, 1],
       [0, 2, 0],
       [0, 3, 0],
       [0, 4, 1],
       [0, 5, 0],
       [0, 6, 1]])

In [5]:
def rotate(ox,oy, px, py):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    j=1 if np.random.random() < 0.5 else -1
    
    qx = ox + math.cos(j*math.pi/2) * (px - ox) - math.sin(j*math.pi/2) * (py - oy)
    qy = oy + math.sin(j*math.pi/2) * (px - ox) + math.cos(j*math.pi/2) * (py - oy)
    return (qx, qy)

In [27]:
def random_rot(polymer):
    random_number = np.random.randint(1,int(polymer.shape[0])-2)
    
#     print(random_number)
    new_polymer = polymer.copy()
    
    qx,qy = rotate(polymer[random_number,0].astype(float), polymer[random_number,1].astype(float),polymer[random_number +1:,0].astype(float),polymer[random_number +1:,1].astype(float))
    
    new_polymer[random_number + 1:, 0] = qx 
    new_polymer[random_number + 1:, 1] = qy
    
#     print(new_polymer)
    #for i in range(new_polymer.shape[0]-1):
    #    if (new_polymer[i,:2] in new_polymer[:i,:2]) and (new_polymer[i,:2] in new_polymer[i+1:,:2]):
    #        print("double detected")
    #        return polymer
#     print("Returned new polymer")
    return new_polymer
    
    
    
    

In [7]:
#plt.scatter(*a[:,:2].astype(float).T, color = a[:,2])
a1=random_rot(a)
print(f"THIS IS a1 {a1}")
# plt.scatter(*a1[:,:2].astype(float).T, color = a1[:,2])

2
THIS IS a1 [[0 1 1]
 [0 2 0]
 [0 3 0]
 [1 3 1]
 [2 3 0]
 [3 3 1]]


In [8]:
a2=random_rot(a1)
print(f"THIS IS a1 {a1}")
# plt.scatter(*a2[:,:2].astype(float).T, color = a2[:,2])

3
THIS IS a1 [[0 1 1]
 [0 2 0]
 [0 3 0]
 [1 3 1]
 [2 3 0]
 [3 3 1]]


expected values after rotation should be an int because since when we do 90 degrees rotation, then it should fall on the grid of the axis which is an int

In [9]:
#compute energies of a2
a2

array([[0, 1, 1],
       [0, 2, 0],
       [0, 3, 0],
       [1, 3, 1],
       [1, 2, 0],
       [1, 1, 1]])

In [10]:
#initial energy is 0
energy = 0

#since we are only computing for the energies for H-H interactions, so only loop through that, i.e, label = 1:
#get all distances between H-H, and if dist = 1, then they are "in contact",a nd energy decreases by -1
#we only consider dist = 1, since we only consider nearest neighbors: up, down, left, right - no diagonals
for index,(x1,y1,label) in enumerate(a2):
    if label == 1:
        print('print', x1, y1, label)
        for (x2,y2,label) in a2[index+1:]:
            if label == 1:
                print('print2', x2,y2,label)
                #get euclidean distance
                dist = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
                print('dist', dist)
                if dist == 1.0:
                    energy = energy - 1
                

print 0 1 1
print2 1 3 1
dist 2.23606797749979
print2 1 1 1
dist 1.0
print 1 3 1
print2 1 1 1
dist 2.0
print 1 1 1


In [11]:
energy

-1

putting energy to function

In [12]:
def compute_energies(state):
    energy = 0
    for index,(x1,y1,label) in enumerate(state):
        if label == 1:
            for (x2,y2,label) in state[index+1:]:
                if label == 1:
                    #get euclidean distance
                    dist = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
                    if dist == 1.0:
                        energy = energy - 1
    return energy  

In [23]:
def delta_energy(old_state, new_state):
    #added the two energies since compute energies will give a value of negative numbers if there is a H-H interaction
    energy_old_state = compute_energies(old_state)
    energy_new_state = compute_energies(new_state)
    return energy_old_state + energy_new_state

In [24]:
def metropolis_step(state):
    #initialize new_configuration
    new_configuration = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0], [0,0,0]])
    
    #get new_state: use random rotate function
    new_state = random_rot(state)
    #calculate energy:
    delta_e = delta_energy(state, new_state)
    print('delta_e', delta_e)
    #condition: if delta_e <= 0, accept the new configuration, if not, accept the new configuration only with probability
    #p = exp(-beta*delta_e)
    if delta_e <= 0:
        print('accept_new')
        new_configuration = new_state
    else:
        prob = np.exp(-beta*delta_e)
        print('prob', prob)
        unif = np.random.rand()
        print('unif', unif)
        if unif < prob:  
            print('accept_new')
            new_configuration = new_state
        else:
            print('old_state')
            new_configuration = state
    return new_configuration

In [28]:
#arrangement of elements in each list: (x_coord, y_coord, label: H = 1, P = 0)
#6 mer sequence from example: HPPHPH
state = np.array([[0,1,1],[0,2,0],[0,3,0],[0,4,1],[0,5,0], [0,6,1]])

In [29]:
n_realizations = 10
beta = 10
print('orig state', state)
for i in range(n_realizations):   
    print('i')
    state = metropolis_step(state)
    energy_state = compute_energies(state)
    print('energy_state', energy_state)
    if  energy_state == -2:
        print('After', i ,'metropolis steps, the desired new_configuration having minimum energy is obtained')
        final_state = state
        break

print('final_state', final_state)

orig state [[0 1 1]
 [0 2 0]
 [0 3 0]
 [0 4 1]
 [0 5 0]
 [0 6 1]]
i
delta_e 0
accept_new
energy_state 0
i
delta_e 0
accept_new
energy_state 0
i
delta_e 0
accept_new
energy_state 0
i
delta_e -2
accept_new
energy_state -2
After 3 metropolis steps, the desired new_configuration having minimum energy is obtained
final_state [[0 1 1]
 [0 2 0]
 [0 1 0]
 [1 1 1]
 [1 0 0]
 [1 0 1]]
